In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, DoubleType, DataType, BooleanType, TimestampType
from pyspark.sql import SparkSession

In [0]:
spark.conf.set("fs.azure.account.auth.type.databriconnect.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.databriconnect.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.databriconnect.dfs.core.windows.net", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.databriconnect.dfs.core.windows.net", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.databriconnect.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
display(dbutils.fs.ls("abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT"))

path,name,size,modificationTime
abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT/Address/,Address/,0,1746462767000
abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT/Customer/,Customer/,0,1746462768000
abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT/CustomerAddress/,CustomerAddress/,0,1746462768000
abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT/Product/,Product/,0,1746462769000
abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT/ProductCategory/,ProductCategory/,0,1746462769000
abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT/ProductDescription/,ProductDescription/,0,1746462771000
abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT/ProductModel/,ProductModel/,0,1746462768000
abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT/ProductModelProductDescription/,ProductModelProductDescription/,0,1746462767000
abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT/SalesOrderDetail/,SalesOrderDetail/,0,1746462768000
abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT/SalesOrderHeader/,SalesOrderHeader/,0,1746462767000


In [0]:
table_name = []

for i in dbutils.fs.ls("abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT"):
    table_name.append(i.name[:-1])

print(table_name)


['Address', 'Customer', 'CustomerAddress', 'Product', 'ProductCategory', 'ProductDescription', 'ProductModel', 'ProductModelProductDescription', 'SalesOrderDetail', 'SalesOrderHeader']


In [0]:
for i in table_name:
    path = 'abfss://bronze@databriconnect.dfs.core.windows.net/SalesLT/' + i + '/' + i + '.parquet'
    df = spark.read.format("parquet").load(path)
    columns = df.columns

    for col in columns:
        if "Date" in col or 'date' in col:
            df = df.withColumn(col,date_format(from_utc_timestamp(df[col].cast(TimestampType()),"UTC"),"yyyy-MM-dd"))
    
    output_path = 'abfss://silver@databriconnect.dfs.core.windows.net/SalesLT/' + i + '/'
    df.write.format("delta").mode("overwrite").save(output_path)

